# Decode keypoints from neural data

In [11]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append("../../")

import pyaldata as pyal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

from tools.reports.report_initial import run_initial_report
from tools.params import Params, colors
from tools.dsp.preprocessing import preprocess
import tools.viz.mean_firing as firing
import tools.viz.dimensionality as dim
import tools.viz.rrr as viz_rrr
import tools.viz.utilityTools as vizutils
import tools.decoding.rrr as rrr

import tools.decoding.decodeTools as decutils
import tools.dataTools as dt

import torch





The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
data_dir = "/data/bnd-data/raw/"
session = 'M062_2025_03_21_14_00'

df = pyal.load_pyaldata(data_dir + session[:4] + '/' + session)
df = preprocess(df, only_trials=False)

field values_before_camera_trigger could not be converted to int.
field idx_before_camera_trigger could not be converted to int.
array field all_KSLabel could not be converted to int.
array field SSp_KSLabel could not be converted to int.
array field VAL_KSLabel could not be converted to int.
array field CP_KSLabel could not be converted to int.
array field MOp_KSLabel could not be converted to int.
field values_Sol_direction could not be converted to int.
field idx_Sol_direction could not be converted to int.
field values_Sol_duration could not be converted to int.
field idx_Sol_duration could not be converted to int.
field idx_sol_on could not be converted to int.
array field all_KSLabel could not be converted to int.
array field SSp_KSLabel could not be converted to int.
array field VAL_KSLabel could not be converted to int.
array field CP_KSLabel could not be converted to int.
array field MOp_KSLabel could not be converted to int.
field values_Sol_direction could not be converted t

/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/utils.py:155: UserWarning: values_MotSen1_X might be a time-varying field. It matches the length of all_spikes on 99.85875706214689% of trials
  warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/utils.py:155: UserWarning: idx_MotSen1_X might be a time-varying field. It matches the length of all_spikes on 99.85875706214689% of trials
  warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/utils.py:155: UserWarning: values_MotSen1_Y might be a time-varying field. It matches the length of all_spikes on 99.85875706214689% of trials
  warnings.warn(
/home/me24/.cache/pypoetry/virtualenvs/earthquake-analysis-Z8riI-lB-py3.12/src/PyalData/pyaldata/utils.py:155: UserWarning: idx_MotSen1_Y might be a time-varying field. It matches the length of all_spikes on 99.85875706214689% of tr

Combined every 3 bins
Resulting all_spikes ephys data shape is (NxT): (23, 16000)
Resulting SSp_spikes ephys data shape is (NxT): (66, 16000)
Resulting VAL_spikes ephys data shape is (NxT): (122, 16000)
Resulting CP_spikes ephys data shape is (NxT): (280, 16000)
Resulting MOp_spikes ephys data shape is (NxT): (179, 16000)


In [ ]:
arr = dt.get_data_array(
    data_list=[pyal.select_trials(df, df.trial_name == 'trial')],
    area='MOp',
    epoch=Params.perturb_epoch_long
)
n_sessions, n_targets, n_trials, n_time, n_comp = arr.shape
data = arr.reshape((n_targets*n_trials, n_time, n_comp))
data.shape

def get_data_from_df(df):
        arr = dt.get_data_array(
        data_list=[pyal.select_trials(df, df.trial_name == 'trial')],
        area='MOp',
        epoch=Params.perturb_epoch_long
    )
    n_sessions, n_targets, n_trials, n_time, n_comp = arr.shape
    data = arr.reshape((n_targets*n_trials, n_time, n_comp))
    return data

/home/me24/repos/earthquake-analysis/notebooks/pipelines/../../tools/dataTools.py:88: UserWarning: Dropping the trials with the following IDs because of invalid time indexing. For more information, try warn_per_trial=True

        [706]
  df_ = pyal.restrict_to_interval(data_list[0], epoch_fun=epoch)
/home/me24/repos/earthquake-analysis/notebooks/pipelines/../../tools/dataTools.py:98: UserWarning: Dropping the trials with the following IDs because of invalid time indexing. For more information, try warn_per_trial=True

        [706]
  df_ = pyal.restrict_to_interval(df, epoch_fun=epoch) if epoch is not None else df


(264, 134, 10)

In [28]:
df_trials = pyal.select_trials(df, df.trial_name == 'trial').iloc[:-1]
# kinematics = np.concatenate()

In [32]:
angles = []
for col in df_trials.columns:
    if col.endswith("angle"):
        angles.append(df_trials[f'{col}'].values)
angles = np.array(angles)

In [34]:
len(df_trials)

352

In [35]:
len(pyal.select_trials(df, df.trial_name == 'trial'))

353

In [22]:
from sklearn.model_selection import KFold


kf = KFold(n_splits=5, shuffle=False)

# Get indices for the first fold
for train_idx, test_idx in kf.split(data):
    print("Train indices:", train_idx.shape)
    print("Test indices:", test_idx.shape)
    break  # just show the first fold

Train indices: (211,)
Test indices: (53,)


In [ ]:
# Using the first fold
train_data, train_labels = data[train_idx], labels[train_idx]
test_data, test_labels = data[test_idx], labels[test_idx]

# Create datasets
train_dataset = TrialDataset(train_data, train_labels)
test_dataset = TrialDataset(test_data, test_label